In [1]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_name = 'all_parts'

data_file = f'../{data_name}_vectors_drags_lifts.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)

#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

/home/xli/anaconda3/envs/surrogate_autogluon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_505,dim_506,dim_507,dim_508,dim_509,dim_510,dim_511,dim_512,drag,lift
406,-1.334886,0.321034,0.117262,-0.305855,0.444947,1.023753,-0.451729,-1.019958,-0.258471,0.228218,...,2.593884,0.411334,-0.664937,-0.254676,-1.019561,-1.698375,2.505520,-1.301861,0.395,0.425
54,-0.109114,-0.027087,-0.042780,-0.158241,0.175974,-0.289997,-0.133499,0.108182,-0.338404,-0.450839,...,0.087662,1.532475,-0.305243,-0.534035,-0.571023,0.098019,-0.941778,-1.245735,0.645,0.885
241,-0.101703,0.144499,-0.023952,-0.053210,0.061378,-0.311319,0.125281,-0.027643,0.149108,0.443060,...,1.002401,1.859596,-0.137056,0.355251,2.653296,-2.105825,0.281476,-0.948454,0.446,0.756
952,-0.226892,-0.005032,-0.167621,0.388806,0.103854,-0.436283,0.040336,0.165606,0.180177,-0.430575,...,1.288276,0.013573,-2.236531,-0.609770,0.127713,-0.095409,-0.348951,1.393792,0.384,0.177
337,0.185246,-0.069290,0.026386,-0.056042,-0.145993,0.035420,-0.000915,0.042858,0.043498,0.346347,...,0.753645,2.592806,-2.071527,2.080894,-0.331741,-0.813147,-0.123172,-1.497938,0.381,0.074


In [2]:
label = 'lift'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    837.000000
mean       0.378389
std        0.238650
min        0.002000
25%        0.184000
50%        0.344000
75%        0.540000
max        0.988000
Name: lift, dtype: float64


In [3]:
import os
save_path = f'./agModels-{data_name}_{label}'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

bag_folds = 5 #suggestion range [5, 10]
bag_sets = 3 #suggestion range [1, 20]
stack_levels = 3 #suggestion range [0, 3]
metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, 
                                                                                  presets='best_quality', 
                                                                                  auto_stack="True", 
                                                                                  num_bag_folds=bag_folds, 
                                                                                  num_bag_sets=bag_sets,
                                                                                  num_stack_levels=stack_levels,
                                                                                  verbosity=4)

Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': 'True',
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'verbosity': 4}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': 'True',
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=5, num_bag_sets=3
Saving ./agModels-all_parts_lift/learner.pkl
Saving ./agModels-all_parts_lift/predi

In [4]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_504,dim_505,dim_506,dim_507,dim_508,dim_509,dim_510,dim_511,dim_512,drag
1046,1.619915,-0.103578,0.073575,-1.121774,0.141612,0.590395,-0.228947,-0.219177,-0.779382,-0.057121,...,0.823198,1.352653,-0.688813,0.550425,-2.302893,0.768312,-1.411428,0.085539,-0.680079,0.527
657,0.271968,-0.005573,-0.117678,-0.265639,-0.109453,-0.103864,0.155861,-0.004339,0.145107,-0.075331,...,2.078918,1.211526,-0.097052,-0.075910,0.071608,0.316292,-1.772259,0.027680,1.245017,0.363
127,0.463927,-0.125690,-0.007975,0.209698,0.010536,0.418671,-0.035722,-0.130384,-0.453760,0.582740,...,1.008988,2.737970,0.340102,0.494566,0.096378,0.921504,0.511855,0.058074,0.371419,0.316
951,0.851177,-0.137168,-0.108522,0.558352,0.014424,-0.032274,0.004961,-0.087621,0.066688,-0.268336,...,2.797498,1.963243,0.650137,-0.393522,1.317100,-0.996707,0.200376,0.762205,0.870829,0.413
134,0.028853,0.104246,-0.015825,-0.181903,-0.039073,-0.342067,0.125156,0.060900,0.152334,-0.304059,...,0.934917,1.027455,0.964021,-1.136593,-0.250099,0.149763,-0.103794,-1.079714,2.322213,0.495


In [5]:
# %%capture log_output
# %config InlineBackend.figure_format = 'retina'
# %config Application.log_level = 'DEBUG'
# %config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

# with open('./output_all_parts.log', 'w') as f:
#     f.write(log_output.stdout)

Loading: ./agModels-all_parts_lift/predictor.pkl
Loading: ./agModels-all_parts_lift/learner.pkl
Loading: ./agModels-all_parts_lift/models/trainer.pkl
Loading: ./agModels-all_parts_lift/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/ExtraTreesMSE_BAG

0.38605546951293945
0.333299458026886
0.22752366960048676
0.5284650325775146
0.24350759387016296
0.36746925115585327
0.33097901940345764
0.6479768753051758
0.21160858869552612
0.24776268005371094
0.41631022095680237
0.3211703598499298
0.4074740707874298
0.326324462890625
0.3088242709636688
0.41315487027168274
0.45403754711151123
0.3286348283290863
0.7304797172546387
0.6219326853752136
0.22224204242229462
0.46452000737190247
0.2955702245235443
0.3601243793964386
0.15709568560123444
0.5274313688278198
0.4473847448825836
0.3806151747703552
0.28092432022094727
0.5846222639083862
0.2132113128900528
0.32566240429878235
0.526969313621521
0.30459466576576233
0.3380029797554016
0.453060507774353
0.45352134108543396
0.2195570021867752
0.2470744103193283
0.21114952862262726
0.2884680926799774
0.5964096784591675
0.23855429887771606
0.31955721974372864
0.5520036220550537
0.5997314453125
0.27027103304862976
0.26894187927246094
0.5699574947357178
0.5879573822021484
0.6385743618011475
0.33891242742538

/home/xli/anaconda3/envs/surrogate_autogluon/lib/python3.10/site-packages/autogluon/core/utils/plots.py:138: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')
Loading: ./agModels-all_parts_lift/models/KNeighborsUnif_BAG_L1/model.pkl


*** End of fit() summary ***
{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN', 'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN', 'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB', 'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB', 'RandomForestMSE_BAG_L1': 'StackerEnsembleModel_RF', 'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost', 'ExtraTreesMSE_BAG_L1': 'StackerEnsembleModel_XT', 'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular', 'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost', 'NeuralNetTorch_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch', 'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB', 'WeightedEnsemble_L2': 'WeightedEnsembleModel', 'LightGBMXT_BAG_L2': 'StackerEnsembleModel_LGB', 'LightGBM_BAG_L2': 'StackerEnsembleModel_LGB', 'RandomForestMSE_BAG_L2': 'StackerEnsembleModel_RF', 'CatBoost_BAG_L2': 'StackerEnsembleModel_CatBoost', 'ExtraTreesMSE_BAG_L2': 'StackerEnsembleModel_XT', 'NeuralNetFastAI_BAG_L2': 'StackerEnsembleModel_NNFastAiT

Loading: ./agModels-all_parts_lift/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/WeightedEnsemble_L2/model.pkl
Loading: ./agModels-all_parts_lift/models/LightGBMXT_BAG_L2/model.pkl
Loading: ./agModels-all_parts_lift/models/LightGBM_BAG_L2/model.pkl
Loading: ./agModels-all_parts_lift/models/RandomForestMSE_BAG_L2/model.pk

                     model  score_test  score_val  pred_time_test  \
0          LightGBM_BAG_L4   -0.191326  -0.187760       30.072493   
1        LightGBMXT_BAG_L4   -0.191972  -0.187485       30.302885   
2        LightGBMXT_BAG_L2   -0.192004  -0.184079       12.254820   
3      WeightedEnsemble_L3   -0.192344  -0.183734       17.586892   
4           XGBoost_BAG_L2   -0.192593  -0.188222       10.334434   
5      WeightedEnsemble_L5   -0.192600  -0.185524       33.895618   
6           XGBoost_BAG_L4   -0.192897  -0.189075       28.249757   
7   RandomForestMSE_BAG_L4   -0.192953  -0.191190       28.123271   
8   RandomForestMSE_BAG_L2   -0.193326  -0.187684       10.251105   
9          LightGBM_BAG_L2   -0.193367  -0.185764       12.144261   
10     WeightedEnsemble_L4   -0.193455  -0.185223       23.011994   
11  NeuralNetFastAI_BAG_L4   -0.193502  -0.193432       28.477594   
12       LightGBMXT_BAG_L3   -0.193561  -0.188026       21.071831   
13    LightGBMLarge_BAG_L4   -0.19

In [6]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  regression
AutoGluon identified the following types of features:
('float', []) : 513 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]


In [7]:
train_data_pred = predictor.predict(train_data)
test_data_pred = predictor.predict(test_data)

import numpy as np
#save np array y_train_hat to a csv file
np.savetxt(f'./{data_name}_vectors_y_test_hat_{label}.csv', test_data_pred, delimiter=',')
np.savetxt(f'./{data_name}_vectors_y_train_hat_{label}.csv', train_data_pred, delimiter=',')

Loading: ./agModels-all_parts_lift/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-all_parts_lift/models/ExtraTreesMSE_BAG_L2/model.pkl
Loading: ./agModels-all_parts_lift/models/LightGBMLarge_BAG_L2/model.pkl
Loading: ./agModels-all_parts_lift/models/LightGBMXT_BAG_L2/mod